In [121]:
import kerchunk.hdf
import fsspec
import xarray as xr

In [118]:
fs = fsspec.filesystem(
    protocol="az",
    account_name="saenigmaarchivedev",
    anon=False,
)

In [119]:
urls = list(fs.glob("analysis/uch/nwp/*/*/*.nc"))

In [143]:
datasets = []
for url in urls[:3]:
    with fs.open(url) as f:
        ds = xr.open_dataset(f.read())
        datasets.append(ds)
    print(ds["time_utc"].values)

['2024-09-22T12:00:00.000000000']
['2024-09-22T13:00:00.000000000']
['2024-09-22T14:00:00.000000000']


In [144]:
ds = xr.combine_by_coords(datasets, combine_attrs="override")

/home/uch/.pyenv/versions/3.10.8/envs/pvforecast-etl/lib/python3.10/site-packages/xarray/core/concat.py:544: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  # determine which variables to concatenate


In [145]:
ds = ds.set_coords(["latitude", "longitude"])

In [149]:
ds["latitude"] = ds["latitude"].isel(time_utc=0)
ds["longitude"] = ds["longitude"].isel(time_utc=0)

In [150]:
ds

<xarray.Dataset>
Dimensions:                            (y: 1606, x: 1906, time_utc: 3,
                                        calculation_time_utc: 1)
Coordinates:
    latitude                           (y, x) float64 39.67 39.68 ... 62.67
    longitude                          (y, x) float64 334.6 334.6 ... 40.07
  * calculation_time_utc               (calculation_time_utc) datetime64[ns] ...
  * time_utc                           (time_utc) datetime64[ns] 2024-09-22T1...
Dimensions without coordinates: y, x
Data variables:
    temperature_K                      (time_utc, calculation_time_utc, y, x) float32 ...
    wind_u_m_s                         (time_utc, calculation_time_utc, y, x) float32 ...
    wind_v_m_s                         (time_utc, calculation_time_utc, y, x) float32 ...
    accumulated_direct_radiation_W_m2  (time_utc, calculation_time_utc, y, x) float32 ...
    accumulated_global_radiation_W_m2  (time_utc, calculation_time_utc, y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ekmi
    GRIB_centreDescription:  Copenhagen
    GRIB_subCentre:          255
    Conventions:             CF-1.7
    institution:             Copenhagen
    history:                 2024-09-24T11:19 GRIB to CDM+CF via cfgrib-0.9.1...
    parameter:               temperature

In [103]:
xr.open_dataset("local_run.nc", engine="h5netcdf")

<xarray.Dataset>
Dimensions:                            (calculation_time_utc: 1, y: 1606,
                                        x: 1906, time_utc: 1)
Coordinates:
  * calculation_time_utc               (calculation_time_utc) datetime64[ns] ...
    time_utc                           (time_utc) datetime64[ns] 2024-09-24T1...
Dimensions without coordinates: y, x
Data variables:
    latitude                           (y, x) float64 ...
    longitude                          (y, x) float64 ...
    temperature_K                      (time_utc, calculation_time_utc, y, x) float32 ...
    wind_u_m_s                         (time_utc, calculation_time_utc, y, x) float32 ...
    wind_v_m_s                         (time_utc, calculation_time_utc, y, x) float32 ...
    accumulated_direct_radiation_W_m2  (time_utc, calculation_time_utc, y, x) float32 ...
    accumulated_global_radiation_W_m2  (time_utc, calculation_time_utc, y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ekmi
    GRIB_centreDescription:  Copenhagen
    GRIB_subCentre:          255
    Conventions:             CF-1.7
    institution:             Copenhagen
    history:                 2024-09-25T07:09 GRIB to CDM+CF via cfgrib-0.9.1...
    parameter:               temperature

In [104]:
import kerchunk.hdf

singles = []
for url in urls[:3]:
    with fs.open(url) as f:
        chunks = kerchunk.hdf.SingleHdf5ToZarr(f, url)
        singles.append(chunks.translate())
    print("Done:", url)

Done: analysis/uch/nwp_hdf5/model_run_time_utc=20240923T090000Z/time_utc=20240924T130000Z/20240923T090000Z_20240924T130000Z.nc
Done: analysis/uch/nwp_hdf5/model_run_time_utc=20240923T090000Z/time_utc=20240924T140000Z/20240923T090000Z_20240924T140000Z.nc


In [105]:
from kerchunk.combine import MultiZarrToZarr

In [115]:
mzz = MultiZarrToZarr(
    singles,
    remote_protocol="az",
    remote_options=fs.storage_options,
    concat_dims="time_utc",
    identical_dims=["x", "y"],
)

In [116]:
out = mzz.translate()

AttributeError: 'dict' object has no attribute 'drop_indexes'

In [108]:
import xarray as xr

In [113]:
ds = xr.open_dataset(
    "reference://",
    engine="zarr",
    backend_kwargs=dict(
        storage_options=dict(
            fo=out,
            remote_protocol="az",
            remote_options=fs.storage_options
        ),
        consolidated=False,
    ),
)

In [114]:
ds

<xarray.Dataset>
Dimensions:                            (time_utc: 1, calculation_time_utc: 1,
                                        y: 1606, x: 1906)
Coordinates:
  * calculation_time_utc               (calculation_time_utc) datetime64[ns] ...
  * time_utc                           (time_utc) datetime64[ns] 2024-09-24T1...
Dimensions without coordinates: y, x
Data variables:
    accumulated_direct_radiation_W_m2  (time_utc, calculation_time_utc, y, x) float32 ...
    accumulated_global_radiation_W_m2  (time_utc, calculation_time_utc, y, x) float32 ...
    latitude                           (time_utc, y, x) float64 ...
    longitude                          (time_utc, y, x) float64 ...
    temperature_K                      (time_utc, calculation_time_utc, y, x) float32 ...
    wind_u_m_s                         (time_utc, calculation_time_utc, y, x) float32 ...
    wind_v_m_s                         (time_utc, calculation_time_utc, y, x) float32 ...
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ekmi
    GRIB_centreDescription:  Copenhagen
    GRIB_edition:            2
    GRIB_subCentre:          255
    history:                 2024-09-25T07:09 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             Copenhagen
    parameter:               temperature

In [11]:
import xarray as xr

In [56]:
ds = xr.open_dataset("test.nc")